In [1]:
import pandas as pd
import os
import torch
import sqlite3
from sqlite3 import Error

zdb_path = f'data/EKyn Sleep and Sleep Dep 24-Jun PD35.24-Jun-A1.20240806200742.zdb'

try:
    conn = sqlite3.connect(zdb_path)
except Error as e:
    print(e)

# get recording start stop
cur = conn.cursor()
query = "SELECT value FROM internal_property WHERE key='RecordingStart'"
cur.execute(query)
result = cur.fetchall()
recording_start = int(result[0][0])
query = "SELECT value FROM internal_property WHERE key='RecordingStop'"
cur.execute(query)
result = cur.fetchall()
recording_stop = int(result[0][0])
length_ns = recording_stop - recording_start # ns
length_s = length_ns * 1e-7 # s
hh = length_s // 3600
mm = (length_s % 3600) // 60
ss = ((length_s % 3600) % 60)
print(hh,mm,ss,length_s)
print(recording_start)
print(recording_stop)

71.0 55.0 19.0 258919.0
638585572620000000
638588161810000000


In [13]:
query = "SELECT * FROM scoring_marker WHERE key_id=1"
cur.execute(query)
result = cur.fetchall()
state_mapping = {
    'Sleep-Wake': 2,
    'Sleep-SWS': 1,
    'Sleep-Paradoxical': 0
}
mapped_result = [state_mapping[row[-4]] for row in result]  # Assuming the state is in the 4th-to-last column
len(mapped_result)

25892

In [19]:
import pandas as pd
df = pd.read_csv(f'data/EKyn Sleep and Sleep Dep 24-Jun PD35.24-Jun-A1.20240806200742_y_pred.csv')
df['y_true'] = mapped_result

In [21]:
from sklearn.metrics import classification_report
y_true = df['y_true']
y_pred = df['y_pred']
target_names = ['Sleep-Paradoxical', 'Sleep-SWS', 'Sleep-Wake']
print(classification_report(y_true, y_pred, target_names=target_names))

                   precision    recall  f1-score   support

Sleep-Paradoxical       1.00      1.00      1.00      1176
        Sleep-SWS       1.00      1.00      1.00      9539
       Sleep-Wake       1.00      1.00      1.00     15177

         accuracy                           1.00     25892
        macro avg       1.00      1.00      1.00     25892
     weighted avg       1.00      1.00      1.00     25892

